In [1]:
!venv/Scripts/activate

Error: [Errno 13] Permission denied: 'c:\\Work\\src\\levycampaign\\venv\\Scripts\\python.exe'
'venv' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

In [93]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate


#new_prompt = ChatPromptTemplate(input_variables=['context', 'question'])

template_format = """You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.
                    Please explain the rules based on the context only. If you don't know the answer, say I don't know. \nQuestion: {question} \nContext: {context} \nAnswer:"""

prompt2=ChatPromptTemplate([], input_variables=['context', 'question'], template_format=template_format)

In [89]:
template

ChatPromptTemplate(input_variables=['context', 'question', 'retrieved_context', 'user_query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are a helpful assistant. Use the following context to answer the question. \nQuestion: {question} \nContext: {context} \nAnswer:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['retrieved_context', 'user_query'], input_types={}, partial_variables={}, template='\n### Context:\n{retrieved_context}\n\n### Question:\n{user_query}\n\n### Answer:\n'), additional_kwargs={})])

In [95]:
prompt2

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[])

In [104]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

prompt

c:\Work\src\levycampaign\venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [107]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate

prompt =  PromptTemplate.from_template("""You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.
                    Please explain the rules based on the context only.
                    If you don't know the answer, say I don't know. \nQuestion: {question} \nContext: {context} \nAnswer:""")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.\n                    Please explain the rules based on the context only.\n                    If you don't know the answer, say I don't know. \nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [108]:
print(example_messages[0].content)

You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.
                    Please explain the rules based on the context only.
                    If you don't know the answer, say I don't know. 
Question: filler question 
Context: filler context 
Answer:


In [75]:
#prompt = hub.pull("rlm/rag-prompt")

c:\Work\src\levycampaign\venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [45]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
#prompt = hub.pull("rlm/rag-prompt")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Work\src\levycampaign\venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [112]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001A13F79B010>, search_kwargs={})

In [117]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    print("format_docs")
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

def retriever2(question):
    print("retriever2")
    print(question)
    return "Task decomposition separates molecules into atoms and those into stars"


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain2 = (
    {"context": retriever2, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [121]:
[x for x in ({"context": lambda x: "Task Decomposition decomposes plants into their substance", "question": RunnablePassthrough()} | prompt).stream("What is Task Decomposition?")]

[StringPromptValue(text="You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.\n                    Please explain the rules based on the context only.\n                    If you don't know the answer, say I don't know. \nQuestion: What is Task Decomposition? \nContext: Task Decomposition decomposes plants into their substance \nAnswer:")]

In [110]:
[x for x in ({"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt).stream("What is Task Decomposition?")]

format_docs
[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and ge

[StringPromptValue(text='You are an expert player of Levy & Campaign games, particularly Almoravid. You are helpful and always willing to explain the rules and strategies of the game.\n                    Please explain the rules based on the context only.\n                    If you don\'t know the answer, say I don\'t know. \nQuestion: What is Task Decomposition? \nContext: Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nTree of

In [124]:
for chunk in rag_chain2.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

retriever2
What is Task Decomposition?
I don't know.

In [ ]:
[x for x in ({"context": retriever2, "question": RunnablePassthrough()} | prompt | llm).invoke("What is Task Decomposition?")]

In [135]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

format_docs
[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and ge

In [142]:
array = [i for i in range(1, 11)]

array[:5]


[1, 2, 3, 4, 5]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2801635760.py, line 1)